<a href="https://colab.research.google.com/github/Orloffman/Time-de-Agentes-Pesquisadores-Engenheiros/blob/main/Time_de_Agentes_Pesquisadores_%26_Engenheiros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################

resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################

!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Artigos --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17",
        instruction= """
Você é um assistente de pesquisa acadêmica. Sua tarefa é, a partir do tópico fornecido sobre um resíduo específico, utilizar a ferramenta de busca do Google (Google Search) para encontrar artigos científicos publicados nos últimos 10 anos.
O foco da busca deve ser em:
1.  Aplicações potenciais para o resíduo como matéria-prima para novos materiais.
2.  Métodos de conversão ou processamento deste resíduo para gerar esses novos materiais.

Selecione os 10 artigos mais relevantes com base no fator de impacto da revista e/ou no número de citações. Priorize artigos com maior impacto e citações; artigos com poucas citações podem ser menos confiáveis ou desatualizados.
Artigos que você não consiga definir precisamente a aplicação devem ser desconsiderados e outro artigo deve ser buscado no lugar. Não queremos artigos que sem uma metodologia muito bem definida, desta forma, voce deve ser capaz de
gerar insights sobre cada artigo analisado. Sem esta análise profunda, os passos posteriores serão profundamente prejudicados.

Retorne uma lista contendo, para cada artigo selecionado:
* Título
* Descrição da aplicação proposta (qual produto está sendo desenvolvido)
* Link para o artigo (DOI ou URL, se acessível pela busca)
* Uma breve justificativa (1-2 frases) de por que este artigo é relevante para encontrar aplicações e métodos de processamento do resíduo.
        """,
        description = "Agente que busca artigos no Google",
        tools=[google_search],
    )


    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    artigos = call_agent(buscador, entrada_do_agente_buscador)

    return artigos
    # Executa o agente

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, artigos):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.5-flash-preview-04-17",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""

Você é um pesquisador sênior com vasta experiência acadêmica no desenvolvimento de materiais inovadores a partir de resíduos. Seu papel é analisar detalhadamente cada um dos artigos fornecidos pelo agente_buscador.

Para cada artigo:
1.  Verifique se todas as informações fornecidas pelo agente buscador estão corretamente preenchidas. Caso contrário avise os artigos que estão com informações pendentes.
2.  Detalhe ou busque utilizando o Google (google_search) a metodologia de processamento ou conversão do resíduo utilizada.
3.  Identifique e liste explicitamente os principais parâmetros de processo (ex: temperatura, tempo, proporção de reagentes, tipo de catalisador, etc.) e seus valores ou faixas considerados eficientes ou ótimos no estudo.
4.  Avalie a validade científica dos resultados e a confiabilidade geral do estudo.

Se julgar que um artigo não é cientificamente sólido, relevante para o objetivo, ou confiável, indique claramente o motivo e não prossiga com a análise detalhada para este artigo específico. Foque nos artigos que considera válidos. Se necessário, você pode usar o Google Search para buscar informações contextuais sobre técnicas específicas mencionadas, mas o foco principal é a análise do conteúdo dos artigos fornecidos.

Ao final, para cada artigo analisado e considerado válido, retorne um relatório conciso contendo:
* Objetivo principal do estudo no artigo.
* Resumo da metodologia de processamento/conversão.
* Principais parâmetros de processo identificados e seus valores/faixas.
* Principais resultados e conclusões do artigo.
* Sua avaliação sobre a validade científica e relevância do artigo para o desenvolvimento de uma metodologia de laboratório.

Por fim selecione até 2 aplicações que considerar a mais promissoras para desenvolvimento.
Selecione 1 aplicação para ser um overdelivery em um projeto de pesquisa deste resíduo. Este overdelivery deverá ter uma prioridade de desenvolviemnto menor que as duas aplicações selecionadas.
        """,
        description="Agente que consolida as informações dos artigos ",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nArtigos: {artigos}"
    # Executa o agente
    plano_pesquisado = call_agent(planejador, entrada_do_agente_planejador)
    return plano_pesquisado

In [ ]:
######################################
# --- Agente 3: Redator do Procedimentos --- #
######################################
def agente_redator(topico, plano_pesquisado):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
           Você é um professor universitário e pesquisador experiente, especialista em Engenharia de Materiais, Estatística Aplicada (DOE) e Inovação. Você já publicou centenas de artigos e sua tarefa é elaborar um plano experimental detalhado para testes em escala de laboratório, com base nas 3 aplicações selecionadas pelo Agente 2 (planejador).

O plano deve ser escrito como um procedimento claro e passo a passo, adequado para alunos de iniciação científica com o apoio de técnicos de laboratório.

Cada plano experimental deve incluir:
1.  **Objetivo Geral do Experimento:** O que se pretende investigar ou validar com estes testes? (Ex: "Avaliar a viabilidade da rota X para converter o resíduo Y no material Z e identificar os parâmetros de processo mais influentes em sua eficiência/qualidade").
2.  **Planejamento de Experimentos (DOE) Sugerido:**
    * Com base nos parâmetros de processo e faixas identificados pelo agente_planejador, proponha um DOE inicial.
    * Justifique a escolha do tipo de DOE (ex: fatorial exploratório de triagem, fatorial completo, otimização por superfície de resposta, ou um estudo de um fator por vez se a informação for muito preliminar).
    * Defina claramente:
        * Os fatores (variáveis independentes/parâmetros de processo) a serem estudados.
        * Os níveis para cada fator (valores específicos ou faixas a serem testadas).
        * As respostas (variáveis dependentes) a serem medidas para avaliar o resultado (ex: rendimento da conversão, pureza do material, propriedades mecânicas, etc.).
    * Se os artigos sugerirem múltiplas rotas promissoras, você pode focar na mais recorrente/promissora ou, se pertinente, sugerir um plano principal e alternativas concisas.
3.  **Procedimento Experimental Detalhado (formato de laboratório):**
    * **Materiais e Reagentes:** Lista de todos os materiais, resíduos (com especificação de pré-tratamento, se houver), reagentes químicos (com grau de pureza), etc.
    * **Equipamentos Necessários:** Lista dos equipamentos principais (reatores, fornos, agitadores, medidores, equipamentos de caracterização, etc.) e vidrarias.
    * **Passo a Passo da Execução:** Instruções detalhadas para cada etapa do experimento, incluindo quantidades, sequências de adição, condições de processo (temperatura, tempo, agitação, atmosfera, etc.) para cada corrida experimental definida pelo DOE.
    * **Coleta de Dados e Medições:** Como e quando coletar amostras, quais medições realizar e como quantificar os resultados (ex: "pesar o produto seco", "realizar análise de [TÉCNICA] seguindo o procedimento ou norma aplicável").

4.  **Análise dos Resultados e Critérios de Sucesso:**
    * Como os dados coletados serão analisados (ex: "calcular o rendimento percentual", "comparar as propriedades do material com valores de referência", "análise estatística dos efeitos dos fatores no DOE").
    * Quais tipos de resultados indicariam sucesso, forneceriam insights valiosos ou direcionariam os próximos passos da pesquisa.

            """,
        description="Agente redator de procedimentos experimentais",
        tools=[google_search]
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano pesquisado: {plano_pesquisado}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Engenheiro de Processos sênior em uma empresa de P&D focada em transformar resíduos industriais em novos materiais. Sua prioridade é garantir que os processos sejam simples, padronizados, eficientes e escaláveis. A empresa atualmente realiza experimentos em escala de bancada, mas tem forte ambição de escalar as soluções desenvolvidas.

Sua tarefa é revisar os planejamentos experimentais e o procedimentos de laboratório propostos pelo agente_redator. Avalie os seguintes aspectos:

1.  **Executabilidade em Laboratório:**
    * O procedimento é claro, completo e suficientemente detalhado para ser executado por alunos de IC com apoio técnico?
    * Os equipamentos e materiais listados são comumente disponíveis ou razoavelmente acessíveis em um laboratório de pesquisa de materiais?
    * Existem etapas excessivamente complexas, perigosas (além do razoável e sem mitigação clara) ou propensas a erros para uma escala de bancada?
    * O plano permite a padronização e reprodutibilidade dos experimentos?

2.  **Viabilidade de Escalonamento:**
    * O processo proposto utiliza matérias-primas, reagentes ou condições que seriam proibitivamente caros, raros ou difíceis de manusear em escala industrial?
    * Os equipamentos ou técnicas descritos para o laboratório têm análogos industriais viáveis?
    * Identifique potenciais gargalos, desafios de segurança, ambientais ou de controle de qualidade que surgiriam ao tentar escalar este processo.
    * Comente sobre a complexidade geral do processo em relação ao potencial de escalonamento e sua possível viabilidade financeira preliminar em maior escala (sem necessidade de cálculos detalhados, mas baseado em princípios de engenharia de processos).

**Output:**
* Se o planejamento e o procedimento forem considerados adequados, retorne-os com uma nota de aprovação, destacando seus pontos fortes para execução e escalabilidade.
* Caso identifique problemas ou áreas de melhoria, aponte-os claramente e sugira modificações específicas no plano experimental ou no procedimento para torná-los mais práticos, seguros, padronizáveis ou com maior potencial de escalonamento.

            """,
        description="Agente revisor de planejamentos experimentais.",
        tools=[google_search]
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de busca de metodologias para experiemntos em laboratório com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o resíduos sobre o qual você quer buscar metodologias para valorizar: ")

# Inserir lógica do sistema de agentes ################################################


if not topico:
  print("Esqueceu de digitar o resíduo")

else:
  print(f"Vamos criar metodologias para trabalhar com {topico}")

  artigos = agente_buscador(topico, data_de_hoje)
  print("\n--- RESULTADO DO AGENTE 1 (BUSCADOR) ---\n")
  display(to_markdown(artigos))
  print("----------------------------------------------------------------------")

  plano_pesquisado = agente_planejador(topico, artigos)
  print("\n--- RESULTADO DO AGENTE 2 (PLANEJADOR) ---\n")
  display(to_markdown(plano_pesquisado))
  print("----------------------------------------------------------------------")

  rascunho = agente_redator(topico, plano_pesquisado)
  print("\n--- RESULTADO DO AGENTE 3 (REDATOR) ---\n")
  display(to_markdown(rascunho))
  print("----------------------------------------------------------------------")

  texto_revisado = agente_revisor(topico, rascunho)
  print("\n--- RESULTADO DO AGENTE 4 (REVISOR) ---\n")
  display(to_markdown(texto_revisado))
  print("----------------------------------------------------------------------")



In [ ]:
# Juntando os resultados dos agentes

doc_completo = f"""
Parte 1: Resultados do Buscador

{artigos}
--------------------------------------------------------------------
Parte 2: Resultados do Planejador

{plano_pesquisado}
--------------------------------------------------------------------
Parte 3: Resultados do Redator

{rascunho}
--------------------------------------------------------------------
Parte 4: Resultados do Revisor

{texto_revisado}

Fim do relatório
"""

  # Agora a variável texto_completo_para_documento contém tudo!
display(to_markdown(doc_completo))


In [ ]:
!pip install markdown xhtml2pdf

In [ ]:
import markdown
from xhtml2pdf import pisa # Pisa é o componente da xhtml2pdf que faz a conversão
# from google.colab import files # Para o download no final


In [ ]:
#Gerando um arquivo PDF

#Instalar a biblioteca no Colab

!pip install fpdf2

In [ ]:
texto_completo_markdown = doc_completo

# Converter o texto Markdown para HTML
html_content = markdown.markdown(texto_completo_markdown)

# Função para converter HTML para PDF
def convert_html_to_pdf(source_html, output_filename):
    # Abrir o arquivo de saída em modo binário de escrita
    with open(output_filename, "w+b") as result_file:
        # Converter HTML para PDF
        pisa_status = pisa.CreatePDF(
                source_html,                # string de HTML
                dest=result_file)           # objeto de arquivo

    # Retorna true se a conversão foi bem-sucedida
    return not pisa_status.err

# Definir o nome do arquivo PDF de saída
nome_arquivo_pdf_formatado = "relatorio_agentes_formatado.pdf"

# Chamar a função de conversão
success = convert_html_to_pdf(html_content, nome_arquivo_pdf_formatado)

if success:
    print(f"Arquivo PDF '{nome_arquivo_pdf_formatado}' criado com sucesso com formatação!")
    # Para baixar o arquivo (descomente a linha abaixo e a importação de 'files' no início)
    # from google.colab import files
    # files.download(nome_arquivo_pdf_formatado)
else:
    print("Ocorreu um erro ao criar o PDF.")

In [ ]:
from google.colab import files

files.download("relatorio_agentes_formatado.pdf")

In [ ]:
#Gerando um Arquivo Word (.docx)
#Instalar a biblioteca no Colab
!pip install python-docx

In [ ]:
from google.colab import files

files.download("relatorio_agentes_gemini.docx")

In [ ]:
#Convertendo Markdown para HTML e depois HTML para DOCX

!pip install markdown html2docx



In [ ]:
#Código para converter
import markdown

from html2docx import html2docx
from docx import Document as PythonDocxDocument


html_content = markdown.markdown(texto_completo_markdown, extensions=['extra'])

docx_bytes_io = html2docx(html_content, title="Relatório Gerado")

nome_arquivo_word_html_formatado = "relatorio_agentes_via_html.docx"

with open(nome_arquivo_word_html_formatado, "wb") as f:
    f.write(docx_bytes_io.getvalue())

print(f"Arquivo Word '{nome_arquivo_word_html_formatado}' criado com sucesso via HTML!")

In [ ]:
from google.colab import files

files.download("relatorio_agentes_via_html.docx")